#Регрессия

##Базовая модель

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

DATA_DIR = Path("/content/drive/MyDrive/Учеба/Фреймворки/data")
SIMPLE_REG_PATH = DATA_DIR / "regression_data_simple.csv"

reg_simple = pd.read_csv(SIMPLE_REG_PATH)

print("Simple regression dataset shape:", reg_simple.shape)
print(reg_simple.head())

feature_cols = ["TotalQuantity", "NumLines", "MeanPrice"]
target_col = "TotalCheck"

X = reg_simple[feature_cols].values
y = reg_simple[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Baseline Linear Regression (simple features, no scaling) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")

print("\nCoefficients (feature -> weight):")
for name, coef in zip(feature_cols, lin_reg.coef_):
    print(f"{name}: {coef:.4f}")
print("Intercept:", lin_reg.intercept_)


Simple regression dataset shape: (39574, 6)
   Invoice  CustomerID  TotalCheck  TotalQuantity  NumLines  MeanPrice
0   489434     13085.0      505.30            166         8   4.081250
1   489435     13085.0      145.80             60         4   2.625000
2   489436     13078.0      630.33            193        19   3.730526
3   489437     15362.0      310.75            145        23   3.628261
4   489438     18102.0     2286.24            826        17   2.591176

Feature matrix shape: (39574, 3)
Target vector shape: (39574,)

Train size: 31659
Test size: 7915

=== Baseline Linear Regression (simple features, no scaling) ===
MAE:  215.55
RMSE: 605.22
R²:   0.6301

Coefficients (feature -> weight):
TotalQuantity: 0.7245
NumLines: 6.6625
MeanPrice: 7.5319
Intercept: 93.43906975346869


Базовая линейная регрессия на трёх агрегированных признаках (TotalQuantity, NumLines, MeanPrice) показала среднее качество: MAE ≈ 215.55, RMSE ≈ 605.22,
𝑅
2
≈
0.63
R
2
≈0.63. Модель улавливает общий рост суммы чека с увеличением количества и числа позиций, но заметно уступает KNN по точности. В дальнейшем для улучшения результата целесообразно протестировать удаление выбросов по TotalCheck, масштабирование признаков, логарифмирование целевой переменной и расширение набора признаков за счёт временных и производных (нелинейных) признаков.

##Улучшенная модель

После базового эксперимента с простой линейной регрессией на трёх агрегированных признаках (TotalQuantity, NumLines, MeanPrice) имеет смысл проверить, удастся ли улучшить качество модели за счёт более подробного описания корзины и аккуратного препроцессинга признаков.

Для этого используется улучшенный датасет regression_data.csv, в котором для каждого чека сформированы:

агрегированные признаки корзины:
TotalQuantity, NumLines, NumUniqueItems, MeanPrice;

временные признаки:
InvoiceHour, InvoiceDayOfWeek, InvoiceMonth;

бинарный признак IsNewCustomer, отражающий, является ли покупатель новым по отношению к рассматриваемому периоду;

набор признаков формата qty_<…> для top-N наиболее часто встречающихся в чеках товаров, где каждый столбец показывает количество единиц соответствующего товара в данном чеке.

В отличие от базового варианта, при обучении линейной модели на расширенном наборе признаков:

числовые признаки масштабируются с помощью StandardScaler, что улучшает численную устойчивость оценки коэффициентов и позволяет корректно учитывать признаки с разными диапазонами значений;

категориальный признак Country кодируется методом One-Hot Encoding (OneHotEncoder), чтобы информация о стране была учтена в модели без введения искусственного порядка между категориями;

служебные поля (Invoice, CustomerID, InvoiceDate) не участвуют в обучении и используются только как метаданные для идентификации записей.

Такой подход позволяет сопоставить результаты базовой линейной регрессии на простых агрегатах с результатами модели, обученной на более информативном и тщательно подготовленном наборе признаков.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

DATA_DIR = Path("/content/drive/MyDrive/Учеба/Фреймворки/data")
REG_PATH = DATA_DIR / "regression_data_hard.csv"

reg_imp = pd.read_csv(REG_PATH)

print("Improved regression dataset shape:", reg_imp.shape)
print(reg_imp.head())

target_col = "TotalCheck"
exclude_cols = ["Invoice", "CustomerID", "InvoiceDate", target_col]

feature_cols = [c for c in reg_imp.columns if c not in exclude_cols]

print("\nNumber of feature columns:", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_imp[feature_cols]
y = reg_imp[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

lin_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LinearRegression()),
    ]
)

lin_pipe.fit(X_train, y_train)

y_pred = lin_pipe.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Linear Regression (improved feature set + scaling + Country OHE) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Improved regression dataset shape: (39574, 113)
   Invoice  CustomerID         Country          InvoiceDate  TotalCheck  \
0   491725     12346.0  United Kingdom  2009-12-14 08:34:00        45.0   
1   491742     12346.0  United Kingdom  2009-12-14 11:00:00        22.5   
2   491744     12346.0  United Kingdom  2009-12-14 11:02:00        22.5   
3   492718     12346.0  United Kingdom  2009-12-18 10:47:00        22.5   
4   492722     12346.0  United Kingdom  2009-12-18 10:55:00         1.0   

   TotalQuantity  NumLines  NumUniqueItems  MeanPrice  InvoiceHour  ...  \
0             10         1               1        4.5            8  ...   
1              5         1               1        4.5           11  ...   
2              5         1               1        4.5           11  ...   
3              5         1               1        4.5           10  ...   
4              1         1               1        1.0           10  ...   

   qty_ANTIQUE_SILVER_TEA_GLASS_ETCHED  qty_HANGIN

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


target_col = "TotalCheck"

q_high = reg_imp[target_col].quantile(0.995)
mask = reg_imp[target_col] <= q_high
reg_trim = reg_imp[mask].reset_index(drop=True)

print("Original rows:", len(reg_imp))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

exclude_cols = ["Invoice", "CustomerID", "InvoiceDate", target_col]
feature_cols = [c for c in reg_trim.columns if c not in exclude_cols]

print("\nNumber of feature columns:", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_trim[feature_cols]
y = reg_trim[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

lin_pipe_trim = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LinearRegression()),
    ]
)

lin_pipe_trim.fit(X_train, y_train)

y_pred = lin_pipe_trim.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Linear Regression (improved features, trimmed target + scaling + Country OHE) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Number of feature columns: 109
Example feature columns: ['Country', 'TotalQuantity', 'NumLines', 'NumUniqueItems', 'MeanPrice', 'InvoiceHour', 'InvoiceDayOfWeek', 'InvoiceMonth', 'IsNewCustomer', 'qty_DOORMAT_RED_RETROSPOT']

Feature matrix shape: (39376, 109)
Target vector shape: (39376,)

Train size: 31500
Test size: 7876

=== Linear Regression (improved features, trimmed target + scaling + Country OHE) ===
MAE:  115.99
RMSE: 229.30
R²:   0.8306


После удаления выбросов по целевой переменной TotalCheck (обрезка по 99.5-му перцентилю) и обучения линейной регрессии на расширенном наборе признаков с масштабированием числовых столбцов и One-Hot кодированием Country качество модели существенно улучшилось: MAE ≈ 115.99, RMSE ≈ 229.30,
𝑅
2
≈
0.83
R
2
≈0.83. Модель стала значительно точнее по сравнению с базовой линейной регрессией и сопоставима по качеству с лучшими вариантами KNN. Далее имеет смысл проверить, как на результат повлияет использование регуляризованных линейных моделей (Ridge, Lasso, ElasticNet).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


q_high = reg_imp[target_col].quantile(0.995)
mask = reg_imp[target_col] <= q_high
reg_trim = reg_imp[mask].reset_index(drop=True)

print("Original rows:", len(reg_imp))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

exclude_cols = ["Invoice", "CustomerID", "InvoiceDate", target_col]
feature_cols = [c for c in reg_trim.columns if c not in exclude_cols]

print("\nNumber of feature columns:", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_trim[feature_cols]
y = reg_trim[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

def eval_model(model, name):
    pipe = Pipeline(
        steps=[
            ("preprocess", preprocessor),
            ("model", model),
        ]
    )
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print(f"\n=== {name} ===")
    print(f"MAE:  {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²:   {r2:.4f}")
    return pipe

lin_model = eval_model(LinearRegression(), "LinearRegression (baseline, trimmed + scaling + OHE)")

ridge_model = eval_model(Ridge(alpha=1.0), "Ridge (alpha=1.0)")
ridge_model_01 = eval_model(Ridge(alpha=0.1), "Ridge (alpha=0.1)")
ridge_model_10 = eval_model(Ridge(alpha=10.0), "Ridge (alpha=10.0)")

lasso_model = eval_model(Lasso(alpha=0.001, max_iter=10000), "Lasso (alpha=0.001)")
lasso_model_01 = eval_model(Lasso(alpha=0.01, max_iter=10000), "Lasso (alpha=0.01)")
lasso_model_1 = eval_model(Lasso(alpha=0.1, max_iter=10000), "Lasso (alpha=0.1)")

enet_model = eval_model(ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=10000), "ElasticNet (alpha=0.001, l1_ratio=0.5)")
enet_model_01 = eval_model(ElasticNet(alpha=0.01, l1_ratio=0.5, max_iter=10000), "ElasticNet (alpha=0.01, l1_ratio=0.5)")
enet_model_1 = eval_model(ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000), "ElasticNet (alpha=0.1, l1_ratio=0.5)")


Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Number of feature columns: 109
Example feature columns: ['Country', 'TotalQuantity', 'NumLines', 'NumUniqueItems', 'MeanPrice', 'InvoiceHour', 'InvoiceDayOfWeek', 'InvoiceMonth', 'IsNewCustomer', 'qty_DOORMAT_RED_RETROSPOT']

Feature matrix shape: (39376, 109)
Target vector shape: (39376,)

Train size: 31500
Test size: 7876

=== LinearRegression (baseline, trimmed + scaling + OHE) ===
MAE:  115.99
RMSE: 229.30
R²:   0.8306

=== Ridge (alpha=1.0) ===
MAE:  116.01
RMSE: 229.26
R²:   0.8307

=== Ridge (alpha=0.1) ===
MAE:  115.99
RMSE: 229.29
R²:   0.8306

=== Ridge (alpha=10.0) ===
MAE:  116.55
RMSE: 229.82
R²:   0.8298


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.904e+07, tolerance: 1.051e+06
  model = cd_fast.enet_coordinate_descent(



=== Lasso (alpha=0.001) ===
MAE:  115.99
RMSE: 229.29
R²:   0.8306

=== Lasso (alpha=0.01) ===
MAE:  115.97
RMSE: 229.20
R²:   0.8307

=== Lasso (alpha=0.1) ===
MAE:  116.18
RMSE: 229.35
R²:   0.8305

=== ElasticNet (alpha=0.001, l1_ratio=0.5) ===
MAE:  116.82
RMSE: 230.09
R²:   0.8294

=== ElasticNet (alpha=0.01, l1_ratio=0.5) ===
MAE:  117.90
RMSE: 231.09
R²:   0.8279

=== ElasticNet (alpha=0.1, l1_ratio=0.5) ===
MAE:  118.67
RMSE: 229.98
R²:   0.8296


На усечённом датасете (обрезка по 99.5-му перцентилю TotalCheck) с масштабированием признаков и One-Hot кодированием Country базовая линейная регрессия показала качество MAE ≈ 115.99, RMSE ≈ 229.30, R2≈0.83. Добавление регуляризации в виде Ridge, Lasso и ElasticNet не привело к заметному улучшению метрик: все варианты дают очень близкие значения MAE/RMSE/R², а наилучший результат демонстрирует Lasso с α=0.01 (MAE ≈ 115.97, RMSE ≈ 229.20,
𝑅2≈0.83). Таким образом, регуляризованные модели в данном случае не улучшают качество по сравнению с обычной линейной регрессией и могут рассматриваться скорее как средство лёгкой стабилизации и упрощения модели, чем как способ получить принципиально более высокую точность.

Далее проверяется гипотеза о том, что для линейной регрессии может быть достаточно агрегированных, временных признаков и флага IsNewCustomer, а детализированные признаки формата qty_* лишь раздувают размерность без существенного прироста качества. Для этого из улучшенного датасета удаляются все столбцы qty_*, а модель повторно обучается на усечённом по таргету наборе признаков (с сохранением масштабирования и One-Hot кодирования Country).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


target_col = "TotalCheck"

q_high = reg_imp[target_col].quantile(0.995)
mask = reg_imp[target_col] <= q_high
reg_trim = reg_imp[mask].reset_index(drop=True)

print("Original rows:", len(reg_imp))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

exclude_base = ["Invoice", "CustomerID", "InvoiceDate", target_col]
qty_cols = [c for c in reg_trim.columns if c.startswith("qty_")]
exclude_cols = exclude_base + qty_cols

feature_cols = [c for c in reg_trim.columns if c not in exclude_cols]

print("\nNumber of feature columns (no qty_*):", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_trim[feature_cols]
y = reg_trim[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

lin_pipe_no_qty = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LinearRegression()),
    ]
)

lin_pipe_no_qty.fit(X_train, y_train)

y_pred = lin_pipe_no_qty.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Linear Regression (trimmed target, no qty_*, scaling + Country OHE) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Number of feature columns (no qty_*): 9
Example feature columns: ['Country', 'TotalQuantity', 'NumLines', 'NumUniqueItems', 'MeanPrice', 'InvoiceHour', 'InvoiceDayOfWeek', 'InvoiceMonth', 'IsNewCustomer']

Feature matrix shape: (39376, 9)
Target vector shape: (39376,)

Train size: 31500
Test size: 7876

=== Linear Regression (trimmed target, no qty_*, scaling + Country OHE) ===
MAE:  149.87
RMSE: 310.62
R²:   0.6891


После удаления признаков формата qty_* и использования только агрегатов корзины, временных признаков, IsNewCustomer и Country качество линейной регрессии заметно снизилось. Это показывает, что для линейной модели детализированное описание состава корзины через qty_* даёт существенный вклад в качество, и их полное исключение приводит к заметной потере информации о структуре чека.

##Имплементирование алгоритма

Ниже приведена самостоятельная реализация алгоритма линейной регрессии на «обычных» данных, сформированных для базового бейзлайна: в качестве признаков используются агрегаты по чеку (TotalQuantity, NumLines, MeanPrice), в качестве целевой переменной ‒ TotalCheck. Модель обучается без масштабирования признаков и без дополнительной обработки выбросов, что позволяет напрямую сравнить результаты кастомной реализации с базовой библиотечной моделью.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


class MyLinearRegression:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        if X.ndim != 2:
            raise ValueError("X must be 2D array of shape (n_samples, n_features)")
        if y.ndim != 1:
            raise ValueError("y must be 1D array of shape (n_samples,)")

        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of samples")

        if self.fit_intercept:
            ones = np.ones((X.shape[0], 1))
            X_ext = np.hstack([ones, X])
        else:
            X_ext = X

        XtX = X_ext.T @ X_ext
        XtX_pinv = np.linalg.pinv(XtX)
        Xt_y = X_ext.T @ y
        w = XtX_pinv @ Xt_y

        if self.fit_intercept:
            self.intercept_ = w[0]
            self.coef_ = w[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = w

        return self

    def predict(self, X):
        if self.coef_ is None:
            raise RuntimeError("You must call fit() before predict().")

        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(1, -1)

        y_hat = X @ self.coef_
        if self.fit_intercept:
            y_hat = y_hat + self.intercept_
        return y_hat


print("Simple regression dataset shape:", reg_simple.shape)
print(reg_simple.head())

feature_cols = ["TotalQuantity", "NumLines", "MeanPrice"]
target_col = "TotalCheck"

X = reg_simple[feature_cols].values
y = reg_simple[target_col].values

print("\nFeature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

my_lin = MyLinearRegression(fit_intercept=True)
my_lin.fit(X_train, y_train)

y_pred = my_lin.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Custom Linear Regression (simple features, no scaling) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")

print("\nCoefficients (feature -> weight):")
for name, coef in zip(feature_cols, my_lin.coef_):
    print(f"{name}: {coef:.4f}")
print("Intercept:", my_lin.intercept_)


Simple regression dataset shape: (39574, 6)
   Invoice  CustomerID  TotalCheck  TotalQuantity  NumLines  MeanPrice
0   489434     13085.0      505.30            166         8   4.081250
1   489435     13085.0      145.80             60         4   2.625000
2   489436     13078.0      630.33            193        19   3.730526
3   489437     15362.0      310.75            145        23   3.628261
4   489438     18102.0     2286.24            826        17   2.591176

Feature matrix shape: (39574, 3)
Target vector shape: (39574,)

Train size: 31659
Test size: 7915

=== Custom Linear Regression (simple features, no scaling) ===
MAE:  215.55
RMSE: 605.22
R²:   0.6301

Coefficients (feature -> weight):
TotalQuantity: 0.7245
NumLines: 6.6625
MeanPrice: 7.5319
Intercept: 93.43906975326118


Собственная реализация линейной регрессии (MyLinearRegression) на простом наборе признаков (TotalQuantity, NumLines, MeanPrice) показала те же метрики, что и модель LinearRegression из библиотеки sklearn, а также совпадающие значения коэффициентов и свободного члена. Это подтверждает корректность имплементации алгоритма и позволяет использовать кастомную модель для сравнения с библиотечной реализацией.

Для сравнения с библиотечной моделью LinearRegression была реализована кастомная версия линейной регрессии и применена к улучшенному бейзлайну: используются агрегированные, временные и товарные признаки (qty_*), признак IsNewCustomer, а также закодированный One-Hot признак Country. Целевая переменная TotalCheck предварительно усечена по 99.5-му перцентилю, числовые признаки масштабируются, категориальный признак кодируется, после чего модель обучается на полученной матрице признаков.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


class MyLinearRegression:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        if X.ndim != 2:
            raise ValueError("X must be 2D array of shape (n_samples, n_features)")
        if y.ndim != 1:
            raise ValueError("y must be 1D array of shape (n_samples,)")

        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of samples")

        if self.fit_intercept:
            ones = np.ones((X.shape[0], 1))
            X_ext = np.hstack([ones, X])
        else:
            X_ext = X

        XtX = X_ext.T @ X_ext
        XtX_pinv = np.linalg.pinv(XtX)
        Xt_y = X_ext.T @ y
        w = XtX_pinv @ Xt_y

        if self.fit_intercept:
            self.intercept_ = w[0]
            self.coef_ = w[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = w

        return self

    def predict(self, X):
        if self.coef_ is None:
            raise RuntimeError("You must call fit() before predict().")

        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(1, -1)

        y_hat = X @ self.coef_
        if self.fit_intercept:
            y_hat = y_hat + self.intercept_
        return y_hat

target_col = "TotalCheck"

q_high = reg_imp[target_col].quantile(0.995)
mask = reg_imp[target_col] <= q_high
reg_trim = reg_imp[mask].reset_index(drop=True)

print("Original rows:", len(reg_imp))
print("Trimmed rows:", len(reg_trim))
print("Cutoff (99.5% quantile) TotalCheck:", q_high)

exclude_cols = ["Invoice", "CustomerID", "InvoiceDate", target_col]
feature_cols = [c for c in reg_trim.columns if c not in exclude_cols]

print("\nNumber of feature columns:", len(feature_cols))
print("Example feature columns:", feature_cols[:10])

X = reg_trim[feature_cols]
y = reg_trim[target_col].values

print("\nRaw feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

numeric_features = [c for c in feature_cols if c != "Country"]
categorical_features = ["Country"] if "Country" in feature_cols else []

transformers = []
if numeric_features:
    transformers.append(("num", StandardScaler(), numeric_features))
if categorical_features:
    transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features))

preprocessor = ColumnTransformer(transformers=transformers)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

print("\nTrain size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

print("Processed train shape:", X_train_proc.shape)
print("Processed test shape:", X_test_proc.shape)

my_lin_imp = MyLinearRegression(fit_intercept=True)
my_lin_imp.fit(X_train_proc, y_train)

y_pred = my_lin_imp.predict(X_test_proc)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Custom Linear Regression (improved baseline: trimmed target + scaling + full features) ===")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")


Original rows: 39574
Trimmed rows: 39376
Cutoff (99.5% quantile) TotalCheck: 5739.156000000027

Number of feature columns: 109
Example feature columns: ['Country', 'TotalQuantity', 'NumLines', 'NumUniqueItems', 'MeanPrice', 'InvoiceHour', 'InvoiceDayOfWeek', 'InvoiceMonth', 'IsNewCustomer', 'qty_DOORMAT_RED_RETROSPOT']

Raw feature matrix shape: (39376, 109)
Target vector shape: (39376,)

Train size: 31500
Test size: 7876
Processed train shape: (31500, 151)
Processed test shape: (7876, 151)

=== Custom Linear Regression (improved baseline: trimmed target + scaling + full features) ===
MAE:  115.99
RMSE: 229.30
R²:   0.8306


Кастомная реализация линейной регрессии на улучшенном бейзлайне (усечённый по 99.5-му перцентилю таргет, масштабирование числовых признаков, One-Hot кодирование Country, агрегированные и товарные признаки) показала метрики, полностью совпадающие с результатами библиотечной модели LinearRegression при аналогичном препроцессинге, что подтверждает корректность собственной имплементации алгоритма на расширенном наборе признаков.

#Классификация

##Базовая модель

##Улучшенная модель

##Имплементирование алгоритма

###Базовая модель

###Улучшенная модель